In [1]:
import pandas as pd

allowed_langs = ["en", "ru", "de", "fr", "uk-UA", "zh"]

In [2]:
messages = pd.read_json(
    path_or_buf="./oasst_ready/2023-04-12_oasst_ready.messages.jsonl.gz",
    lines=True,
)

In [3]:
messages[:3]

,message_id,user_id,created_date,text,role,lang,review_count,review_result,deleted,synthetic,emojis,labels,detoxify,message_tree_id,tree_state,parent_id,rank
0,6ab24d72-0181-4594-a9cd-deaf170242fb,c3fe8c76-fc30-4fa7-b7f8-c492f5967d18,2023-02-05T14:23:50.983374+00:00,Can you write a short introduction about the r...,prompter,en,3,1.0,False,False,"{'+1': 10, '_skip_reply': 1, '_skip_ranking': 4}","{'spam': {'value': 0.0, 'count': 3}, 'lang_mis...","{'toxicity': 0.00044308538781400006, 'severe_t...",6ab24d72-0181-4594-a9cd-deaf170242fb,ready_for_export,NaN,NaN
1,c8e83833-ecbc-44fe-b6db-735228c25a1c,2c96e467-66f0-4be7-9693-bda51356a424,2023-02-06T13:50:44.657083+00:00,"""Monopsony"" refers to a market structure where...",assistant,en,3,1.0,False,False,"{'+1': 3, '_skip_labeling': 1}","{'spam': {'value': 0.0, 'count': 3}, 'fails_ta...","{'toxicity': 0.000263965659542, 'severe_toxici...",6ab24d72-0181-4594-a9cd-deaf170242fb,ready_for_export,6ab24d72-0181-4594-a9cd-deaf170242fb,0.0
2,6708c47f-05c9-4346-b3d2-40b2bd24fde4,2c96e467-66f0-4be7-9693-bda51356a424,2023-02-06T18:48:49.391686+00:00,Now explain it to a dog,prompter,en,3,1.0,False,False,NaN,"{'spam': {'value': 0.0, 'count': 3}, 'lang_mis...","{'toxicity': 0.036484777927398, 'severe_toxici...",6ab24d72-0181-4594-a9cd-deaf170242fb,ready_for_export,c8e83833-ecbc-44fe-b6db-735228c25a1c,NaN


In [4]:
messages_dict = {}

for i in range(len(messages)):
    mess_id = messages.iloc[i]["message_id"]
    messages_dict[mess_id] = messages.iloc[i].to_dict()

In [5]:
trees = pd.read_json(
    path_or_buf="./oasst_ready/2023-04-12_oasst_ready.trees.jsonl.gz",
    lines=True,
)
trees
trees_dict = {}
for i in range(len(trees)):
    tree_id = trees["message_tree_id"][i]
    trees_dict[tree_id] = trees["prompt"][i]

In [ ]:
example_tree_id = list(trees_dict.keys())[3]

trees_dict[example_tree_id]

In [7]:
filtered_trees_dict = {}

for tree_id in trees_dict:
    tree_lang = trees_dict[tree_id]["lang"]
    # if tree_lang in ["ru"]:
    if tree_lang in ["en"]:
        filtered_trees_dict[tree_id] = trees_dict[tree_id]

In [8]:
len(filtered_trees_dict)

3670

In [11]:
filtered_example_id = list(filtered_trees_dict.keys())[31]
filtered_example = filtered_trees_dict[filtered_example_id]
# filtered_example

In [12]:
from collections.abc import Iterable


def flatten(xs):
    for x in xs:
        if isinstance(x, Iterable) and not isinstance(x, (str, bytes)):
            yield from flatten(x)
        else:
            yield x


# https://codebeautify.org/python-formatter-beautifier - for debug
def get_all_threads(sample):
    # debug
    # initial_message = f"{sample['role'].upper()} {sample['text']} |"
    initial_message = f"{sample['message_id']} "

    def recursive_helper(node, current_path):
        if len(node["replies"]) == 0:
            if node["role"] == "assistant":
                # debug
                # current_path += f"\n|||||||||||||\n {node['role'].upper()} {node['text']}"
                current_path += f" {node['message_id']}"

            return current_path + " <END>"

        current_path += f" {node['message_id']}"
        paths = []
        for reply in node["replies"]:
            if node["labels"]["spam"]["value"] < 0.4:
                path = recursive_helper(reply, current_path)
                paths.append(path)
        return paths

    threads = []
    for reply in sample["replies"]:
        path = recursive_helper(reply, initial_message)
        threads.append(path)

    threads = flatten(threads)
    conversations = []
    for thread in threads:
        if "<END>" in thread:
            # debug
            # pprint.pprint(thread)
            # print(thread.split())
            conversation = thread.split()[:-1]
            conversations.append(conversation)

    return conversations


def ids_to_text(message_ids, messages):
    chat_history = []
    for mess_id in message_ids:
        text = messages[mess_id]["text"]
        chat_history.append(text)

    return chat_history


all_threads = get_all_threads(sample=trees["prompt"][10])
all_threads = get_all_threads(sample=filtered_example)
# all_threads

In [13]:
for thread in all_threads:
    messages_texts = ids_to_text(thread, messages_dict)
    print("\n---\n".join(messages_texts))
    print("-" * 100)

How can I find a girlfriend? I really don't hawe much to offer to others. At least I'm good in computers hardware and Linux. But not much girls have been interested in this kind of things. And it's probably impossible to find cute one.
---
Finding a girlfriend is not about having something to offer, but about being your authentic self and finding someone who appreciates and values you for who you are. Here are a few tips that might help:

Focus on your personal growth: Take some time to work on yourself and become the best version of yourself. This could mean developing your skills and interests, improving your health and wellness, and building your confidence.

Meet new people: Get out of your comfort zone and try to meet new people. You can join clubs or groups that align with your interests, attend events and parties, or simply strike up conversations with people you meet in your daily life.

Be open-minded: Don't limit yourself to just looking for someone who is interested in the s

### prepare for filter export


In [15]:
import pandas as pd

# export_name = "./datasets/filtered_datasets/oasst/ru/oasst_ru_v1.csv"
# export_name = "./filtered/oasst/en/oasst_en_v1.csv"
export_name = ""
messages_count = 0

result_dataset = []

for filtered_id in filtered_trees_dict.keys():
    filtered_tree = filtered_trees_dict[filtered_id]
    filtered_threads_ids = get_all_threads(sample=filtered_tree)
    messages_count += len(filtered_threads_ids)
    for thread in filtered_threads_ids:
        thread_str = " ".join(thread)
        filtered_threads_mess = ids_to_text(thread, messages_dict)
        filtered_threads = "\n=======\n".join(filtered_threads_mess)
        # print(filtered_threads)
        result_dataset.append(
            {
                "conversation": filtered_threads,
                "conversation_ids": thread_str,
            },
        )
        # print("-" * 100)
    # print("-" * 100)
    # print("-" * 100)
    # print("-" * 100)
    # break
# messages_count
pd.DataFrame(data=result_dataset).to_csv(export_name, index=False)

### use labeled dataset


In [5]:
import pandas as pd

dataset_path = "./filtered/oasst/ru/oasst_ru_v1_labeled - oasst_ru_v1.csv"
labeled_dataset = pd.read_csv(dataset_path)
labeled_dataset = labeled_dataset[labeled_dataset["ok/trash"] == "ok"]
conversation_ids = labeled_dataset["conversation_ids"].tolist()
conversation_ids[:3]

['e480f611-0d31-433a-93d2-0e2bc675aa30 d5766ce8-5ea8-4713-aaaf-f8121cc4fb14',
 'e480f611-0d31-433a-93d2-0e2bc675aa30 e22f8f08-2c3d-4e79-9fae-3577ef08485a db585b17-226c-446c-9478-cee5490653f7 21ade148-732b-4f7d-9e5a-8f01a0845b18',
 'e480f611-0d31-433a-93d2-0e2bc675aa30 e22f8f08-2c3d-4e79-9fae-3577ef08485a db585b17-226c-446c-9478-cee5490653f7 fb6e108d-3a83-47a5-903f-5dd258232fbd b57b394a-d07c-4fcf-9cfd-96c060344841 6b7d1313-47ed-407e-8411-d5d1547a892b']

In [12]:
messages = pd.read_json(
    path_or_buf="./oasst_ready/2023-04-12_oasst_ready.messages.jsonl.gz",
    lines=True,
)

messages_dict = {}

for i in range(len(messages)):
    mess_id = messages.iloc[i]["message_id"]
    messages_dict[mess_id] = messages.iloc[i].to_dict()

In [14]:
for conversation_id in conversation_ids:
    ids = conversation_id.split()
    mess = ids_to_text(ids, messages_dict)
    print(mess)
    # break

['Напиши информацию о игре Hytale', 'Hytale - это предстоящая игра-песочница, придуманная Hypixel Studios. Объединяя в себе диапазон игры-песочницы с глубиной ролевой игры, Hytale погружает игроков в живой космос, полный эскапад и творчества.\n\nВ Hytale геймеры могут путешествовать по просторному, процедурно созданному миру и принимать участие в разнообразных действиях. От возведения баз и создания собственных персонажей до создания карт приключений и сражений с монстрами или другими игроками - возможности Hytale безграничны. Игроки также могут создавать свои собственные серверы, которые можно использовать совместно с другими геймерами, что позволяет создавать необычные сообщества.\n\nВ игре также имеется обширная система моддинга, позволяющая игрокам создавать и делиться своими собственными предметами. С помощью системы моддинга геймеры могут создавать пользовательское оружие, броню и другие предметы, а также пользовательские карты и режимы игры. Эта функция позволяет игрокам создава